In [2]:
import pandas as pd
from wbtool import file_misc as fm
from wbtool import use_models
import numpy as np

uselabel = True

if uselabel:
    lbdata = pd.read_csv('/home/wb/samba_dir/cells_brightfield/leadingcell-WB-entirespan/Results_B1ROI1.csv')
    SA = use_models.cv.sam_agent('prompt')
else:
    SA = use_models.cv.sam_agent('auto')

In [ ]:
import matplotlib.pyplot as plt
import proc_data

frame = 0
# img = fm.imgtiff_read('/home/wb/working/celltrack/A4ROI1.tif', frame)
img = np.load('/home/wb/samba_dir/cells_brightfield/leadingcell-WB-entirespan/B1ROI1.npy')[frame]
img = proc_data.value_modify(img)

if uselabel:
    lb = lbdata[lbdata['Slice']==(frame+1)]  # use frame

    ax=plt.gca()
    ax.imshow(img)
    ax.scatter(lb['X'], lb['Y'], color='red', marker='*', s=10)
    plt.show()

In [5]:
if uselabel:
    point_prompt = lb[['X','Y']]
    point_prompt = point_prompt.to_numpy()
    # print(point_prompt)
    # print(img.shape)
    masks = SA.infer(img, point_prompt)
else:
    masks = SA.infer(img)

In [ ]:
from wbtool import show

print(len(point_prompt))
print(point_prompt)

# print(masks[10].keys())
# show.wshow(masks[100]['segmentation'])
# show.wshow(masks[18]['segmentation'])

In [ ]:
from wbtool import show

print(len(masks))
if uselabel:
    tp = masks
else:
    tp = [it['segmentation'] for it in masks]
    tp = tp[1:]

# filter
ma = tp.sum(1).sum(1)
imga = masks.shape[1]*masks.shape[2]
mar = ma/imga
I = mar<0.01
tp = tp[I]
print(len(tp))
print(ma)
ma.sort()
print(ma)
a = show.overlay_mask(img, tp)
show.wshow(a)
# np.save('/home/wb/working/mask_label.npy', masks)